In [1]:
%%capture
!pip install tensorboardX simpletransformers sklearn

In [2]:
!cat /usr/local/cuda/version.txt

!pip install torch==1.6.0
#conda install pytorch==1.1.0 torchvision==0.3.0 cudatoolkit=10.0 -c pytorch
#https://pytorch.org/get-started/previous-versions/

CUDA Version 10.0.130
     |████████████████████████████████| 748.8 MB 5.9 kB/s              


In [3]:
!rm -rf outputs

In [15]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
from tqdm import tqdm
import pandas as pd
df = pd.read_csv("downloads/40k_balanced_pm_acl.csv")
len(df)

24645

In [16]:
df = df[df["source"] != "acl_cambridge"]

In [8]:

# df = df.dropna(subdownloads/pm_abstracts_100k.csv"])
# df["text_nomask"] = df["text"] #backup

# df["text_length"]= df["text"].map(lambda t: len(t.split(" ")))

# df = df[df["text_length"] > 7]


# from octoflow.core import replace_outof_vocab_words, vocab30k

# df["text"] = df["text"].map(lambda t: replace_outof_vocab_words(t, vocab30k))


# df["text"] = df["text"].astype(str)
# df["labels"] = df["labels"].astype(int)
# df.to_csv("downloads/40k_pm_masked.csv", index = False, header=True)

In [26]:
df= df[["text", "labels"]]
df = df.rename(columns={"labels": "label"})

KeyError: "['labels'] not in index"

In [27]:
df =df.dropna()

In [28]:
df.label = df.label.map(int)
df

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,text,label
1997,"While often idiopathic, anterior ischemic opti...",0
1998,Primary cutaneous B-cell lymphomas (PCBCLs) ar...,0
1999,The insufflation of gas during colonoscopy lea...,0
2000,The aim of the present study is to investigate...,0
2001,The aim of this study was to assess predictive...,0
...,...,...
24639,To perform a pilot study on the effect of PRP ...,0
24640,"To analyze success rate, dwell-time, and compl...",0
24641,To compare the efficacy of oral itraconazole v...,0
24642,This controlled trial with II-1 evidence compa...,0


In [29]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df2 = df.sample(frac=1)
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. 
#The first column is the text with type str, and the second column is the label with type int.
size = len(df2)
train_cutoff = round(size*0.8)
train_df= df2[:train_cutoff]
eval_df = df2[train_cutoff:]

model_args = ClassificationArgs(save_steps=-1, save_model_every_epoch=False, sliding_window=True)

model = ClassificationModel('distilbert', 'distilbert-base-uncased',args=model_args, use_cuda=True) # You can set class weights by using the optional weight argument

INFO:filelock:Lock 140057284692120 acquired on /home/ec2-user/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333.lock


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

INFO:filelock:Lock 140057284692120 released on /home/ec2-user/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333.lock
INFO:filelock:Lock 140055085871624 acquired on /home/ec2-user/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

INFO:filelock:Lock 140055085871624 released on /home/ec2-user/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

...

INFO:filelock:Lock 140055086717752 released on /home/ec2-user/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
INFO:filelock:Lock 140055085874928 acquired on /home/ec2-user/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

INFO:filelock:Lock 140055085874928 released on /home/ec2-user/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
INFO:filelock:Lock 140055085874928 acquired on /home/ec2-user/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

INFO:filelock:Lock 140055085874928 released on /home/ec2-user/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


In [30]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)

# Evaluate the model, u
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score )

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/18118 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 18124 features created from 18118 samples.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2266 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/4529 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 4529 features created from 4529 samples.


Running Evaluation:   0%|          | 0/567 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8716333695616227, 'tp': 4244, 'tn': 223, 'fp': 20, 'fn': 42, 'acc': 0.9863104438065798, 'eval_loss': 0.04533659336188967}


In [34]:
!pip install pydash

     |████████████████████████████████| 84 kB 479 kB/s             


1

In [45]:
!pip --version pandas

pip 21.3.1 from /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip (python 3.6)


In [50]:
dfx = pd.read_csv("downloads/biology_85k_s2c.csv",  lineterminator='\n', error_bad_lines=False)
dfx = dfx.dropna(subset=["first_sent"])

In [53]:
dfx

,first_sent,s2Url,doi,journalName,title
1,Osteogenesis imperfecta (OI) is a genetic bone...,https://semanticscholar.org/paper/3ed47627ed36...,10.1002/jbmr.3424,Journal of Bone and Mineral Research,Mutations That Alter the Carboxy‐Terminal‐Prop...
2,SUMMARY We investigated the effects of cyclic ...,https://semanticscholar.org/paper/69ce028dc32b...,10.1242/jeb.00520,Journal of Experimental Biology,Screening of second messengers involved in pho...
3,Background Highly pathogenic avian influenza v...,https://semanticscholar.org/paper/901b049454cb...,10.1371/journal.pone.0044447,PLoS ONE,Protective Efficacy of Newcastle Disease Virus...
4,"Abstract In this paper, an attempt has been ma...",https://semanticscholar.org/paper/a59763c714dd...,10.1016/J.PHYSA.2019.03.009,Physica A: Statistical Mechanics and its Appli...,Viral dynamic model with cellular immune respo...
5,Nos dias atuais existe uma pressao ambientalis...,https://semanticscholar.org/paper/4848ef92a046...,NaN,NaN,Produção de mudas de espécies florestais inocu...
...,...,...,...,...,...
87393,The amphibian chytrid fungus Batrachochytrium ...,https://semanticscholar.org/paper/6e904d7ad9f0...,NaN,NaN,Effect of salts on the deadly amphibian chytri...
87394,There are physiological variations in the leve...,https://semanticscholar.org/paper/59534653ec7a...,10.1046/j.1365-2249.1997.4411460.x,Clinical & Experimental Immunology,Circadian rhythm of leucocytes and lymphocyte ...
87395,1.,https://semanticscholar.org/paper/fe5cdb739902...,10.1139/CJR32-090,Canadian journal of research,NOTE ON AN INVESTIGATION INTO THE FLUORESCENCE...
87396,ABSTRACT We studied nutrient cycling within a ...,https://semanticscholar.org/paper/433e2e7089f2...,10.1080/02705060.2006.9665020,Journal of Freshwater Ecology,Nutrient Cycling in a Natural Stand of Typha a...


In [54]:
dfx["first_sent"] = dfx["first_sent"].astype(str)

In [55]:
len(dfx) #564254

86807

In [10]:
#dfx = dfx[dfx["first_sent"].map(type) == str].reset_index()

In [11]:
#dfx.reset_index().to_csv("downloads/pm_abstracts_100k.csv", index = False, header=True)

In [56]:
def stepwise(step, df, f):
    l = []
    for idx in range(step, len(df)+step-1, step):
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx]
        batch = model.predict(list(sl))
        l = l + list(batch[0])
    return l

In [57]:
predicts = stepwise(10000, dfx["first_sent"], model.predict)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


0 10000


  0%|          | 0/10000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 10000 features created from 10000 samples.


  0%|          | 0/1257 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


80000 90000


  0%|          | 0/6807 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 6807 features created from 6807 samples.


  0%|          | 0/855 [00:00<?, ?it/s]

In [106]:
dt = pd.read_csv("core_pos.csv")

In [181]:

#d[d["first_sent"].str.contains(re.escape("|".join(d["authors"][2:-2].replace("'", "").split(","))).strip().replace('\\', ''))]
def formatStr (x):
    x = eval(x)
    x = [re.escape(v) for v in x]
    return "|".join(x)

regexs = d.authors.apply(formatStr)
matches = pd.Series()
for i, r in tqdm(d.iterrows()):
    matches.loc[i] = len(re.findall(regexs.loc[i], d.first_sent.loc[i])) > 0


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
44962it [00:59, 760.29it/s] 


In [188]:
dc = d[~matches]

In [190]:
dc["authors"] = dc["authors"].map(eval)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [192]:
dc = dc.reset_index()


In [200]:
dc = dc.drop(columns=["fullText"])
dc.to_csv("core_pos_parsed.csv", index = False, header=True)

In [197]:
dc.columns

Index(['index', 'doi', 'coreId', 'oai', 'identifiers', 'title', 'authors',
       'contributors', 'datePublished', 'downloadUrl', 'fullTextIdentifier',
       'pdfHashValue', 'publisher', 'rawRecordXml', 'journals', 'language',
       'relations', 'year', 'topics', 'subjects', 'fullText', 'urls', 'issn',
       'enrichments.references', 'enrichments.citationCount',
       'enrichments.documentType.type', 'enrichments.documentType.confidence',
       'language.code', 'language.name', 'language.id', 'first_sent',
       'predicts'],
      dtype='object')

In [201]:
tt = dc[["first_sent",'doi', 'coreId', 'oai', 'title' ]]

In [203]:
tt

,first_sent,doi,coreId,oai,title
0,Liu and Mertz (Genes Dev.,10.1128/MCB.25.15.6303-6313.2005,3717551,oai:pubmedcentral.nih.gov:1190326,Binding of hnRNP L to the Pre-mRNA Processing ...
1,"Rett syndrome (RTT), a neurodevelopmental diso...",10.1086/382228,3710391,oai:pubmedcentral.nih.gov:1182264,X-Chromosome Inactivation Patterns Are Unbalan...
2,Estimating optimal sample size for microbiolog...,10.1128/AEM.69.10.6174-6178.2003,3286476,oai:pubmedcentral.nih.gov:201250,A Model To Estimate the Optimal Sample Size fo...
3,The antiphospholipid syndrome is a thrombotic ...,NaN,8064782,oai:pubmedcentral.nih.gov:2398067,Antiphospholipid syndrome in a patient with ra...
4,Alcohol dependence (AD) is a complex disorder ...,NaN,8674979,oai:pubmedcentral.nih.gov:3266269,Genome-Wide Association Study of Copy Number V...
...,...,...,...,...,...
42196,Autism spectrum disorder (ASD) is a common neu...,NaN,8273706,oai:pubmedcentral.nih.gov:2662567,The genetic and neurobiologic compass points t...
42197,This paper revisits the problem of the readine...,NaN,8689613,oai:pubmedcentral.nih.gov:3291942,On the Relevance of Using Open Wireless Sensor...
42198,Thyroid cancer is a rare disease.,NaN,8126095,oai:pubmedcentral.nih.gov:2466601,Thyroid carcinoma and its treatment
42199,Hyperosmolar hyperglycaemic state (HHS) is a m...,NaN,8696563,oai:pubmedcentral.nih.gov:3302717,A case of hyperosmolar hyperglycaemic state wi...


In [202]:
tt.to_csv("core_pos_min.csv", index = False, header=True)

In [58]:
dfx["predicts"] = predicts

MASKING raised 2900 to 4100 positive predicts

In [59]:
d = dfx[dfx["predicts"]==1]
d = d[d["first_sent"] != "nan"]



In [62]:
for x in d["first_sent"][:10]:
    print(x)

Osteogenesis imperfecta (OI) is a genetic bone disorder characterized by fractures, low bone mass, and skeletal fragility.
Background Highly pathogenic avian influenza virus (HPAIV) causes a highly contagious often fatal disease in poultry, resulting in significant economic losses in the poultry industry.
Central giant cell granuloma (CGCG) is a reactive bone lesion that occurs mainly in the jaws.
Fasciola hepatica infection continues to be a major problem in the agriculture sector, particularly in sheep and cattle.
O cravo-da-india, Syzygium aromaticum [L] Merr.
Toxoplasmosis is one of the most globally widespread zoonoses with considerable health and economic impacts.
Habitat fragmentation is considered to be one of the greatest threats to biodiversity.
Cnidarian-dinoflagellate symbiosis disruption and subsequent bleaching are major concerns, especially regarding their ecological consequences on coral reefs and temperate coralligenous communities.
Helminths infections caused by Soil-

In [83]:
# inStr = lambda x: any([x.first_sent.startswith(v) for v in x.authors])
# t = pd.Series()
# for i,r in tqdm(d.iterrows()):
#     t.loc[i] = inStr(r)
    

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
44962it [01:09, 643.23it/s]


In [49]:
d

,doi,coreId,oai,identifiers,title,authors,contributors,datePublished,downloadUrl,fullTextIdentifier,...,issn,enrichments.references,enrichments.citationCount,enrichments.documentType.type,enrichments.documentType.confidence,language.code,language.name,language.id,first_sent,predicts
12,NaN,3356872,oai:pubmedcentral.nih.gov:279359,[],ABSENCE OF A CHLORAMPHENICOL-INSENSITIVE PHASE...,"['Plotz, Paul H.', 'Davis, Bernard D.']",[],1962-04,NaN,http://www.pubmedcentral.nih.gov/articlerender...,...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,"Plotz, Paul H. (Harvard Medical School, Boston...",1
16,10.1128/MCB.25.15.6303-6313.2005,3717551,oai:pubmedcentral.nih.gov:1190326,[],Binding of hnRNP L to the Pre-mRNA Processing ...,"['Guang, Shouhong', 'Felthauser, Alicia M.', '...",[],2005-08,NaN,http://www.pubmedcentral.nih.gov/articlerender...,...,0270-7306,[],47.0,NaN,NaN,NaN,NaN,NaN,Liu and Mertz (Genes Dev.,1
21,10.1086/382228,3710391,oai:pubmedcentral.nih.gov:1182264,[],X-Chromosome Inactivation Patterns Are Unbalan...,"['Young, Juan I.', 'Zoghbi, Huda Y.']",[],2004-02-17,NaN,http://www.pubmedcentral.nih.gov/articlerender...,...,0002-9297,[],104.0,NaN,NaN,NaN,NaN,NaN,"Rett syndrome (RTT), a neurodevelopmental diso...",1
42,10.1128/AEM.69.10.6174-6178.2003,3286476,oai:pubmedcentral.nih.gov:201250,[],A Model To Estimate the Optimal Sample Size fo...,"['Altekruse, S. F.', 'Elvinger, F.', 'Wang, Y....",[],2003-10,NaN,http://www.pubmedcentral.nih.gov/articlerender...,...,0099-2240,[],12.0,NaN,NaN,NaN,NaN,NaN,Estimating optimal sample size for microbiolog...,1
47,NaN,8064782,oai:pubmedcentral.nih.gov:2398067,[],Antiphospholipid syndrome in a patient with ra...,"['Kelion, A. D.', 'Cockcroft, J. R.', 'Ritter,...",[],NaN,NaN,http://www.pubmedcentral.nih.gov/articlerender...,...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,The antiphospholipid syndrome is a thrombotic ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144097,NaN,3355414,oai:pubmedcentral.nih.gov:277725,[],ANAEROBIC GROWTH OF PSYCHROPHILIC BACTERIA,"['Upadhyay, J.', 'Stokes, J. L.']",[],1962-02,NaN,http://www.pubmedcentral.nih.gov/articlerender...,...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,"Upadhyay, J. (Washington State University, Pul...",1
1144129,NaN,8689613,oai:pubmedcentral.nih.gov:3291942,[],On the Relevance of Using Open Wireless Sensor...,"['Bagula, Antoine B.', 'Inggs, Gordon', 'Scott...",[],NaN,https://core.ac.uk/download/pdf/8689613.pdf,http://www.pubmedcentral.nih.gov/articlerender...,...,NaN,"[{'date': '2005', 'raw': 'Li, W; Cassandras, C...",NaN,NaN,NaN,en,English,9.0,This paper revisits the problem of the readine...,1
1144131,NaN,8126095,oai:pubmedcentral.nih.gov:2466601,[],Thyroid carcinoma and its treatment,"['Taylor, Selwyn']",[],NaN,NaN,http://www.pubmedcentral.nih.gov/articlerender...,...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,Thyroid cancer is a rare disease.,1
1144146,NaN,8696563,oai:pubmedcentral.nih.gov:3302717,[],A case of hyperosmolar hyperglycaemic state wi...,"['Abd Hamid, Hanisah', 'Umar, Nor-Aini', 'Othm...",[],NaN,https://core.ac.uk/download/pdf/8696563.pdf,http://www.pubmedcentral.nih.gov/articlerender...,...,NaN,"[{'date': '2004', 'raw': 'American Diabetes As...",NaN,NaN,NaN,en,English,9.0,Hyperosmolar hyperglycaemic state (HHS) is a m...,1


In [18]:
for col, r in dfx[dfx["predicts"]==1].iterrows():
    print(r["first_sent"])

The high cervical cancer mortality rate among Latinas compared with other ethnic groups in the United States is of major concern.
The clinical management of cervical cancer in HIV-positive patients has challenges mainly due to the concerns on immune status.
Cervical cancer is almost entirely preventable through vaccination and screening, yet remains one of the 'gravest threats to women's lives' according to the World Health Organization.
Cervical cancer affects half a million women worldwide annually.
Cervical cancer is one of the most common causes of morbidity and mortality among women in developing countries including Ethiopia.
Over the past decade, there are significant advances in the problem of studying the pathogenesis, clinical course and treatment of cervical cancer (CC).
Cervical cancer is a global health challenge in women.
Local advanced cervical cancer (LACC) is a considerable health crisis for women, and neoadjuvant chemotherapy (NACT) followed by radical surgery has been

distilbert
'mcc': 0.7621432780395265, 'tp': 167, 'tn': 480, 'fp': 39, 'fn': 30, 'acc': 0.9036312849162011, 'eval_loss': 0.24669100075132316}

